In [ ]:
!pip install nltk

ERROR: Operation cancelled by user


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
df = pd.read_csv('fake reviews dataset.csv')
df.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [ ]:
print('Shape:',df.shape)

Shape: (40432, 4)


In [ ]:
df.isnull().sum()

,0
category,0
rating,0
label,0
text_,0


In [ ]:
print(df['label'].unique())

['CG' 'OR']


In [ ]:
df['label'].value_counts()

,count
label,
CG,20216
OR,20216


In [ ]:
df['rating'].value_counts()

,count
rating,
5.0,24559
4.0,7965
3.0,3786
1.0,2155
2.0,1967


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df['label'])

In [ ]:
print(y[:10])

[0 0 0 0 0 0 0 0 0 0]


In [ ]:
df.rename(columns={'text_': 'review'}, inplace=True)

In [ ]:
df['category_clean'] = df['category'].apply(lambda x: x.rsplit('_', 1)[0])
df = pd.get_dummies(df, columns=['category_clean'])

In [ ]:
one_hot_cols = [col for col in df.columns if col.startswith('category_clean_')]
for col in one_hot_cols:
    df[col] = df[col].astype(int)

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

df['cleaned_review'] = df['review'].apply(lemmatize_text)


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
df.head()

,category,rating,label,review,category_clean_Books,category_clean_Clothing_Shoes_and_Jewelry,category_clean_Electronics,category_clean_Home_and_Kitchen,category_clean_Kindle_Store,category_clean_Movies_and_TV,category_clean_Pet_Supplies,category_clean_Sports_and_Outdoors,category_clean_Tools_and_Home_Improvement,category_clean_Toys_and_Games
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor...",0,0,0,1,0,0,0,0,0,0
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I...",0,0,0,1,0,0,0,0,0,0
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...,0,0,0,1,0,0,0,0,0,0
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i...",0,0,0,1,0,0,0,0,0,0
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...,0,0,0,1,0,0,0,0,0,0


In [ ]:
df.shape

(40432, 15)

In [ ]:
# Splitting the dataset into two parts:
# X_cat contains the categorical/numerical features by dropping text and target columns
# X_text contains the cleaned text reviews, which will be vectorized separately
X_cat = df.drop(['cleaned_review','review','category','label'], axis=1)
X_text = df['cleaned_review']

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X_train_text, X_test_text, X_train_cat, X_test_cat, y_train, y_test = train_test_split(
    X_text, X_cat, y, test_size=0.3, random_state=42)

In [ ]:
X_train_cat.head()

,rating,category_clean_Books,category_clean_Clothing_Shoes_and_Jewelry,category_clean_Electronics,category_clean_Home_and_Kitchen,category_clean_Kindle_Store,category_clean_Movies_and_TV,category_clean_Pet_Supplies,category_clean_Sports_and_Outdoors,category_clean_Tools_and_Home_Improvement,category_clean_Toys_and_Games
25099,4.0,0,0,0,0,1,0,0,0,0,0
28738,4.0,1,0,0,0,0,0,0,0,0,0
37154,5.0,0,1,0,0,0,0,0,0,0,0
12309,5.0,0,0,0,0,0,1,0,0,0,0
16629,5.0,0,0,0,0,0,0,0,0,1,0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_text_vec = vectorizer.fit_transform(X_train_text)
X_test_text_vec = vectorizer.transform(X_test_text)

# Combine text vectors and categorical features
from scipy.sparse import hstack
# - X_train_final: combined TF-IDF vectors + encoded categorical features
# - y_train: the corresponding labels
X_train_final = hstack([X_train_text_vec, X_train_cat])
X_test_final = hstack([X_test_text_vec, X_test_cat])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
#sets the maximum number of iterations for the algorithm
model = LogisticRegression(max_iter=1000)
#Trains the model
model.fit(X_train_final, y_train)

y_pred = model.predict(X_test_final)
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Intercept: [0.98381573]
Coefficients: [[ 0.18892531  0.23302488  0.01533182 ... -0.21227622 -0.07315806
   0.0440558 ]]
Accuracy: 0.8960428689200329
              precision    recall  f1-score   support

           0       0.91      0.88      0.89      6066
           1       0.88      0.91      0.90      6064

    accuracy                           0.90     12130
   macro avg       0.90      0.90      0.90     12130
weighted avg       0.90      0.90      0.90     12130



In [ ]:
y_pred[:15]

array([0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0])

In [ ]:
y_test[0:15]

array([0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0])

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


model = DecisionTreeClassifier(random_state=42)
model.fit(X_train_final, y_train)
y_pred = model.predict(X_test_final)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7652926628194558
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.77      0.77      6066
           1       0.77      0.76      0.76      6064

    accuracy                           0.77     12130
   macro avg       0.77      0.77      0.77     12130
weighted avg       0.77      0.77      0.77     12130

Confusion Matrix:
 [[4698 1368]
 [1479 4585]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 2. Initialize the Random Forest model
# - n_estimators = number of decision trees -default is 100
# - random_state = ensures reproducible results
model = RandomForestClassifier(n_estimators=100, random_state=42)

# 3. Fit the model on training data

model.fit(X_train_final, y_train)

# 4. Predict the target for the test data
# - X_test_final: test set features (same structure as training)
y_pred = model.predict(X_test_final)

# - Accuracy is the ratio of correct predictions to total predictions
print("Accuracy:", accuracy_score(y_test, y_pred))

# 6. Detailed performance report
# - Shows precision, recall, and F1-score for each class
print("Classification Report:\n", classification_report(y_test, y_pred))

# 7Print confusion matrix to visualize correct vs incorrect classifications
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.8783182192910141
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.91      0.88      6066
           1       0.90      0.85      0.87      6064

    accuracy                           0.88     12130
   macro avg       0.88      0.88      0.88     12130
weighted avg       0.88      0.88      0.88     12130

Confusion Matrix:
 [[5518  548]
 [ 928 5136]]


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 2: Fit Multinomial Naive Bayes on text features only
nb_model = MultinomialNB()
nb_model.fit(X_train_text_vec, y_train)

# Step 3: Predict and evaluate
y_pred_nb = nb_model.predict(X_test_text_vec)

# Step 4: Metrics
print("Accuracy (Naive Bayes):", accuracy_score(y_test, y_pred_nb))
print("Classification Report (Naive Bayes):\n", classification_report(y_test, y_pred_nb))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_nb))

Accuracy (Naive Bayes): 0.8576257213520198
Classification Report (Naive Bayes):
               precision    recall  f1-score   support

           0       0.81      0.93      0.87      6066
           1       0.91      0.79      0.85      6064

    accuracy                           0.86     12130
   macro avg       0.86      0.86      0.86     12130
weighted avg       0.86      0.86      0.86     12130

Confusion Matrix:
 [[5620  446]
 [1281 4783]]
